In [1]:
import os, shutil
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
config = tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.8))
sess = tf.Session(config=config)
KTF.set_session(sess)
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from keras.models import model_from_yaml, load_model
from keras.preprocessing import image 
import numpy as np
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras import layers
from keras import models
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
import numpy as np
from keras.applications import VGG16
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
import random

Using TensorFlow backend.


InternalError: failed initializing StreamExecutor for CUDA device ordinal 0: Internal: failed call to cuDevicePrimaryCtxRetain: CUDA_ERROR_OUT_OF_MEMORY: out of memory; total memory reported: 11721506816

In [19]:
with open('./0cp.yaml') as yamlfile:
    model = model_from_yaml(yamlfile.read())
    model.load_weights('./dogskind_reduce_expand.model.weights.best.hdf5')
from keras import optimizers
# lr: float >= 0. 学习率 Learning rate

# momentum: float >= 0. 参数更新动量 parameter updates momentum

# decay: float >= 0. 学习率每次更新的下降率 Learning rate decay over each update

# nesterov: boolean. 是否应用 Nesterov 动量 whether to apply Nesterov momentum
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [20]:
base_dir = './dogImages'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'valid')
test_dir = os.path.join(base_dir, 'test')
train_plus_dir=os.path.join(base_dir, 'train_plus')
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(256, 256),
        batch_size=40,
        class_mode='categorical')

model.evaluate_generator(test_generator,steps=17, max_queue_size=50, workers=1, use_multiprocessing=True, verbose=1)
#steps需要调试，并非越大越好，通过调试对test集应该取3左右，对valid集应该取5
#（不知道这两个数对其它图像训练是否适用）

Found 836 images belonging to 133 classes.
17/17 [==============================] - 9s 534ms/step


[2.294022286639494, 0.40441176470588236]

In [ ]:
if not os.path.exists(train_reduce_expand_dir):
    shutil.copytree(train_plus_dir,train_reduce_expand_dir)
kind_dic=np.load('labels.npy').item()
for kind in os.listdir(train_reduce_expand_dir):
    pic_kind=kind_dic[kind]
    print (kind)
    kind_dir=os.path.join(train_reduce_expand_dir, kind)
    wrong_num = 0
    for pic in os.listdir(kind_dir):
        if not pic.endswith(".jpg"):
            continue
        pic_dir=os.path.join(kind_dir, pic)
        img = image.load_img(pic_dir, target_size=(256,256))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        result = model.predict_classes(x)
        print ("    ",pic,result[0],pic_kind)
        if pic.startswith('pic_plus') and pic_kind==result[0]:
            print ("    right")
        else:
            print ("    wrong")
            os.remove(pic_dir)
            wrong_num+=1
    parent_pics=[os.path.join(kind_dir, one) for one in os.listdir(kind_dir) if one.endswith(".jpg") and not one.startswith('pic_plus')]
    for pic_dir in range(wrong_num):
        print (pic_dir)
        new_img = load_img(random.choice(parent_pics))
        new_img = img_to_array(new_img)
        new_img = np.expand_dims(new_img, axis=0)
        aug = ImageDataGenerator(rotation_range=30, 
                                 width_shift_range=0.1,
                                 height_shift_range=0.1, 
                                 shear_range=0.1, 
                                 zoom_range=0.2,
                                 horizontal_flip=True, 
                                 fill_mode="nearest")
        imageGen = aug.flow(new_img, batch_size=1, 
                            save_to_dir=kind_dir,
                            save_prefix="pic_plus2", 
                            save_format="jpg")
        for a in imageGen:
            break

In [ ]:
total=0
right=0
kind_dic=np.load('labels.npy').item()
for kind in os.listdir(test_dir):
    kind_total=0
    kind_right=0
    pic_kind=kind_dic[kind]
    print (kind)
    kind_dir=os.path.join(test_dir, kind)
    wrong_num = 0
    for pic in os.listdir(kind_dir):
        if not pic.endswith(".jpg"):
            continue
        pic_dir=os.path.join(kind_dir, pic)
        img = image.load_img(pic_dir, target_size=(256,256))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        result = model.predict_classes(x)
        print ("    ",pic,result[0],pic_kind)
        kind_total+=1
        if pic_kind==result[0]:
            print ("    right")
            kind_right+=1
        else:
            print ("    wrong")
    print("kind_accuracy:",kind_right*1.0/kind_total)
    total+=kind_total
    right+=kind_right
#             os.remove(pic_dir)
#             wrong_num+=1
#     parent_pics=[os.path.join(kind_dir, one) for one in os.listdir(kind_dir) if one.endswith(".jpg") and not one.startswith('pic_plus')]
#     for pic_dir in range(wrong_num):
#         print (pic_dir)
#         new_img = load_img(random.choice(parent_pics))
#         new_img = img_to_array(new_img)
#         new_img = np.expand_dims(new_img, axis=0)
#         aug = ImageDataGenerator(rotation_range=30, 
#                                  width_shift_range=0.1,
#                                  height_shift_range=0.1, 
#                                  shear_range=0.1, 
#                                  zoom_range=0.2,
#                                  horizontal_flip=True, 
#                                  fill_mode="nearest")
#         imageGen = aug.flow(new_img, batch_size=1, 
#                             save_to_dir=kind_dir,
#                             save_prefix="pic_plus2", 
#                             save_format="jpg")
#         for a in imageGen:
#             break
print ("total_accuracy:",right, total,right*1.0/total)